In [2]:
import os
import networkx
import pickle
from namespace.graph.json_processor import get_data_matrix
from namespace.graph.comp_printer import print_comp

In [6]:
with open('./simple_strategy_res/graph.pickle', 'rb') as f:
    G = pickle.load(f)
champion_matrix, trait_matrix, _ = get_data_matrix()

The graph `G` is a `networkx.DiGraph` object. Each of its edge points from a lower level comp to a higher level comp. We can both work from high level to low level or low level to high level. Since high level comps are more important for players, let's make a reverse of the graph.

In [3]:
G_rev = G.reverse()
levels = [5, 6, 7, 8, 9]
subgraph_path = './subgraphs/'
roots = [node[0] for node in G_rev.nodes(data=True) if node[1]['layer'] == len(levels) - 1]
if not os.path.exists(subgraph_path):
    os.mkdir(subgraph_path)
for index, root in enumerate(roots):
    cur_G = networkx.DiGraph()
    champ_names, _, _ = print_comp(root, champion_matrix, trait_matrix)
    cur_G.add_node(str(champ_names), layer=len(levels) - 1)
    descendants = networkx.algorithms.dag.descendants(G_rev, root)
    for bit_code in descendants:
        champ_names, _, _ = print_comp(bit_code, champion_matrix, trait_matrix)
        cur_G.add_node(str(champ_names), layer=levels.index(len(champ_names)))
    for i in cur_G.nodes(data=True):
        for j in cur_G.nodes(data=True):
            if i[1]['layer'] - j[1]['layer'] == 1:
                cur_G.add_edge(i[0], j[0])
    networkx.write_graphml(cur_G, subgraph_path + f'tansition_{index}.graphml')

we save each subgraph to the folder `./subgraphs/` in `.graphml` format, each subgraph represent a transition tree, in this case from level 5 to level 9. Use Cytoscape to visualize it.

In [16]:
with open('./poker_strategy_res/level6.pickle', 'rb') as f:
    level6 = pickle.load(f)

In [17]:
for bit_code in level6:
    champ_names, active_traits, _ = print_comp(bit_code, champion_matrix, trait_matrix)
    print(champ_names)
    print(active_traits)
    print()

['gragas', 'kennen', 'kled', 'lee sin', 'nidalee', 'sejuani']
['brawler: 2', 'cavalier: 2', 'dawnbringer: 2', 'hellion: 2', 'nightbringer: 2', 'skirmisher: 3']

['irelia', "kha'zix", 'olaf', 'pyke', 'riven', 'udyr']
['assassin: 2', 'dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'olaf', 'rakan', 'riven', 'soraka', 'udyr']
['dawnbringer: 2', 'legionnaire: 2', 'renewer: 2', 'sentinel: 3', 'skirmisher: 3']

['gragas', 'kled', 'riven', 'sejuani', 'yasuo', 'ziggs']
['brawler: 2', 'cavalier: 2', 'dawnbringer: 2', 'hellion: 2', 'legionnaire: 2', 'nightbringer: 2']

['aatrox', 'irelia', 'lee sin', 'olaf', 'rakan', 'vladimir']
['legionnaire: 2', 'nightbringer: 2', 'renewer: 2', 'sentinel: 3', 'skirmisher: 3']

['gragas', 'hecarim', 'miss fortune', 'sejuani', 'soraka', 'vladimir']
['brawler: 2', 'cavalier: 2', 'dawnbringer: 2', 'forgotten: 2', 'nightbringer: 2', 'renewer: 2']

['gragas', 'hecarim', 'sejuani', 'soraka', 'thresh', 'vladimir']
['brawler: 2', 'cavalie

['irelia', 'miss fortune', 'olaf', 'riven', 'senna', 'udyr']
['cannoneer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'miss fortune', 'olaf', 'senna', 'thresh', 'udyr']
['cannoneer: 2', 'forgotten: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'nidalee', 'olaf', 'pyke', 'riven', 'udyr']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'nidalee', 'olaf', 'rakan', 'riven', 'udyr']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'nidalee', 'olaf', 'rakan', 'soraka', 'udyr']
['dawnbringer: 2', 'renewer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'nidalee', 'olaf', 'riven', 'senna', 'udyr']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'nidalee', 'pyke', 'rakan', 'riven', 'udyr']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'nidalee', 'pyke', 'rakan', 'soraka', 'udyr']
['dawnbringer: 2', 'renewer: 2', 'sentinel: 3', 'skirmisher:

['ashe', 'gragas', 'hecarim', 'sejuani', 'vayne', 'vladimir']
['brawler: 2', 'cavalier: 2', 'forgotten: 2', 'nightbringer: 2', 'ranger: 2']

['ashe', 'gragas', 'kled', 'sejuani', 'soraka', 'vladimir']
['brawler: 2', 'cavalier: 2', 'dawnbringer: 2', 'nightbringer: 2', 'renewer: 2']

['ashe', 'gragas', 'sejuani', 'soraka', 'varus', 'vladimir']
['brawler: 2', 'dawnbringer: 2', 'nightbringer: 2', 'ranger: 2', 'renewer: 2']

['ashe', 'gragas', 'sejuani', 'soraka', 'vayne', 'vladimir']
['brawler: 2', 'dawnbringer: 2', 'nightbringer: 2', 'ranger: 2', 'renewer: 2']

['ashe', 'hecarim', 'nunu & willump', 'sejuani', 'vayne', 'vladimir']
['brawler: 2', 'cavalier: 2', 'forgotten: 2', 'nightbringer: 2', 'ranger: 2']

['ashe', 'hecarim', 'rakan', 'sejuani', 'vayne', 'vladimir']
['cavalier: 2', 'forgotten: 2', 'nightbringer: 2', 'ranger: 2', 'renewer: 2']

['ashe', 'hecarim', 'sejuani', 'sett', 'vayne', 'vladimir']
['brawler: 2', 'cavalier: 2', 'forgotten: 2', 'nightbringer: 2', 'ranger: 2']

['ashe'

['kennen', 'kled', 'rakan', 'sejuani', 'sett', 'vladimir']
['brawler: 2', 'cavalier: 2', 'hellion: 2', 'nightbringer: 2', 'renewer: 2']

['kennen', 'kled', 'riven', 'sejuani', 'soraka', 'vladimir']
['cavalier: 2', 'dawnbringer: 2', 'hellion: 2', 'nightbringer: 2', 'renewer: 2']

['kennen', 'kled', 'sejuani', 'sett', 'soraka', 'vladimir']
['brawler: 2', 'cavalier: 2', 'hellion: 2', 'nightbringer: 2', 'renewer: 2']

['kennen', 'lee sin', 'olaf', 'pyke', 'rakan', 'vladimir']
['nightbringer: 2', 'renewer: 2', 'sentinel: 3', 'skirmisher: 3']

['kennen', 'lee sin', 'olaf', 'rakan', 'senna', 'vladimir']
['nightbringer: 2', 'renewer: 2', 'sentinel: 3', 'skirmisher: 3']

["kha'zix", 'kled', 'lulu', 'sejuani', 'soraka', 'vladimir']
['cavalier: 2', 'dawnbringer: 2', 'hellion: 2', 'nightbringer: 2', 'renewer: 2']

["kha'zix", 'kled', 'nocturne', 'sejuani', 'soraka', 'vladimir']
['assassin: 2', 'cavalier: 2', 'dawnbringer: 2', 'nightbringer: 2', 'renewer: 2']

["kha'zix", 'kled', 'nunu & willump', 

['gragas', 'irelia', 'nidalee', 'olaf', 'pyke', 'yasuo']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['gragas', 'irelia', 'nidalee', 'olaf', 'rakan', 'yasuo']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['gragas', 'irelia', 'nidalee', 'olaf', 'senna', 'yasuo']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['gragas', 'irelia', 'nunu & willump', 'soraka', 'vladimir', 'yasuo']
['brawler: 2', 'dawnbringer: 2', 'legionnaire: 2', 'nightbringer: 2', 'renewer: 2']

['gragas', 'irelia', 'rakan', 'sejuani', 'soraka', 'yasuo']
['brawler: 2', 'dawnbringer: 2', 'legionnaire: 2', 'nightbringer: 2', 'renewer: 2']

['gragas', 'irelia', 'sejuani', 'soraka', 'vladimir', 'yasuo']
['brawler: 2', 'dawnbringer: 2', 'legionnaire: 2', 'nightbringer: 2', 'renewer: 2']

['gragas', 'irelia', 'sett', 'soraka', 'vladimir', 'yasuo']
['brawler: 2', 'dawnbringer: 2', 'legionnaire: 2', 'nightbringer: 2', 'renewer: 2']

['gragas', 'kalista', 'k

['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf', 'rakan', 'vayne', 'yasuo']
['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf', 'riven', 'senna', 'yasuo']
['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf', 'sejuani', 'senna', 'yasuo']
['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf', 'senna', 'sett', 'yasuo']
['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf', 'senna', 'soraka', 'yasuo']
['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf', 'senna', 'syndra', 'yasuo']
['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf', 'senna', 'thresh', 'yasuo']
['legionnaire: 2', 'nightbringer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'lee sin', 'olaf',

['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['ashe', 'irelia', 'nidalee', 'olaf', 'riven', 'senna']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['brand', 'irelia', 'nidalee', 'olaf', 'riven', 'senna']
['dawnbringer: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['gragas', 'hecarim', "kha'zix", 'miss fortune', 'sejuani', 'senna']
['brawler: 2', 'cannoneer: 2', 'cavalier: 2', 'dawnbringer: 2', 'forgotten: 2']

['gragas', 'hecarim', 'lee sin', 'miss fortune', 'sejuani', 'senna']
['brawler: 2', 'cannoneer: 2', 'cavalier: 2', 'forgotten: 2', 'nightbringer: 2']

['gragas', 'hecarim', 'miss fortune', 'nidalee', 'sejuani', 'senna']
['brawler: 2', 'cannoneer: 2', 'cavalier: 2', 'dawnbringer: 2', 'forgotten: 2']

['gragas', 'hecarim', 'miss fortune', 'riven', 'sejuani', 'senna']
['brawler: 2', 'cannoneer: 2', 'cavalier: 2', 'dawnbringer: 2', 'forgotten: 2']

['gragas', 'irelia', 'kalista', 'nidalee', 'olaf', 'senna']
['dawnbringer: 2',

['hellion: 2', 'legionnaire: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'kennen', 'olaf', 'rakan', 'senna', 'tristana']
['cannoneer: 2', 'hellion: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'kennen', 'olaf', 'rakan', 'soraka', 'tristana']
['hellion: 2', 'renewer: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'kennen', 'olaf', 'sejuani', 'senna', 'tristana']
['cannoneer: 2', 'hellion: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'kennen', 'olaf', 'senna', 'sett', 'tristana']
['cannoneer: 2', 'hellion: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'kennen', 'olaf', 'senna', 'soraka', 'tristana']
['cannoneer: 2', 'hellion: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'kennen', 'olaf', 'senna', 'syndra', 'tristana']
['cannoneer: 2', 'hellion: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', 'kennen', 'olaf', 'senna', 'thresh', 'tristana']
['cannoneer: 2', 'hellion: 2', 'sentinel: 3', 'skirmisher: 3']

['irelia', "kha'zix", 'nidalee', 'olaf', 'pyke', 'tristana']
['assassi